In [5]:
## Convert coordinates into float

import pandas as pd

### property dataset paths
#'../data/raw/bed1.1.csv'
#'../data/raw/bed2.1.csv'
#'../data/raw/bed3.csv'
#'../data/raw/bed4.csv'
#'../data/raw/bed5.csv'
#'../data/raw/featured.csv'
#'../data/raw/newest.1.csv'
#'../data/raw/highest_price.csv'
#'../data/raw/lowest_price.csv'

#Read corresponding property dataset
property_df = pd.read_csv('../data/raw/bed3.csv')
float_coords = []

#Convert each property coordinate from string to float
for prop_coord in property_df["coordinates"]:
    prop_coord2 = prop_coord.split(', ')
    x = float(str(prop_coord2[0][1:]))
    y = float(str(prop_coord2[1][:-1]))
    prop_coord = [x,y]
    float_coords.append(prop_coord)
property_df["coordinates1"] = float_coords

#Display converted coordinates
property_df["coordinates1"]

0             [-37.8647904, 144.9835915]
1             [-37.7048046, 145.1261813]
2             [-37.9966423, 145.1708607]
3             [-37.7384813, 145.0065188]
4             [-37.8433691, 144.6965698]
                     ...                
974           [-37.7632624, 145.0630986]
975           [-37.8396774, 144.9984322]
976           [-37.6058841, 144.9151203]
977           [-37.6984191, 144.9460232]
978    [-38.22845239999999, 145.0367262]
Name: coordinates1, Length: 979, dtype: object

In [225]:
##Calculate distance to closest train station using K-NN

import openrouteservice
import pandas as pd

#Get API key from local device so client can't steal my data
with open('../keys/ors_api_key.txt') as f:
    API_KEY = f.read()

client = openrouteservice.Client(key=API_KEY) # Specify your personal API key

import numpy as np
train_coordinates = []

#Read train station data
df1 = pd.read_excel('../data/raw/Annual_metropolitan_train_station_entries_2020-21.xlsx', engine='openpyxl')
df2 = pd.read_excel('../data/raw/Annual_regional_train_station_entries_2020-21.xlsx', engine='openpyxl')
df=pd.concat([df1,df2])

#Create list containing train coordinates
for index, row in df.iterrows():
    train_coordinates.append([row['Stop_lat'], row['Stop_long']])

#Calculate k=2 closest train stations 
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=2)
knn.fit(train_coordinates)

distance_station = []

#find closest 2 stations for each property
for index, row in property_df.iterrows():
    closest_stations = knn.kneighbors([row["coordinates1"]], return_distance=False) 
    #print(closest_stations)


    closest_dist = 100000
    #for each of the closest stations calculation distance by car using ORS
    for i in closest_stations[0]:
        coords = ((property_df["coordinates1"][index][1], property_df["coordinates1"][index][0]),(train_coordinates[i][1], train_coordinates[i][0]))
        #print(coords)
        routes = client.directions(coords, radiuses=-1)
        curr_dist = routes["routes"][0]["segments"][0]["distance"]
        if curr_dist < closest_dist:
            closest_dist = curr_dist
    
    #Add to closest distance to list
    distance_station.append(closest_dist)

6353.0
5257.3
13357.5
12874.0
5870.9
10783.4
21523.1
26942.6
3576.5
3053.2
22787.2
20569.9
9269.6
10687.9
3315.8
7574.2
59878.4
65852.2
1321.7
1708.8


Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f61d2dcbaf0>
Traceback (most recent call last):
  File "/usr/lib/python3.8/weakref.py", line 103, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [184]:
#Calculate closest Hospital using K-NN

import openrouteservice
import pandas as pd

#Get API key from local device so client can't steal my data
with open('../keys/ors_api_key.txt') as f:
    API_KEY = f.read()

client = openrouteservice.Client(key=API_KEY) # Specify your personal API key

import numpy as np
hospital_coordinates = []
#Read hospital data
df1 = pd.read_excel('../data/raw/Annual_metropolitan_train_station_entries_2020-21.xlsx', engine='openpyxl')
df2 = pd.read_excel('../data/raw/Annual_regional_train_station_entries_2020-21.xlsx', engine='openpyxl')
df=pd.concat([df1,df2])

#Create list containing hospital coordinates
for index, row in df.iterrows():
    hospital_coordinates.append([row['Stop_lat'], row['Stop_long']])

from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=2)
knn.fit(hospital_coordinates)

#Calculate k=2 closest train stations 
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=2)
knn.fit(hospital_coordinates)

distance_hospital = []

#find closest 2 hospitals for each property
for index, row in property_df.iterrows():
    closest_hospitals = knn.kneighbors([row["coordinates1"]], return_distance=False) \

    closest_dist = 100000
    #for each of the closest stations calculation distance by car using ORS
    for i in closest_hospitals[0]:
        coords = ((property_df["coordinates1"][index][1], property_df["coordinates1"][index][0]),(hospital_coordinates[i][1], hospital_coordinates[i][0]))
        #print(coords)
        routes = client.directions(coords, radiuses=-1)
        curr_dist = routes["routes"][0]["segments"][0]["distance"]
        if curr_dist < closest_dist:
            closest_dist = curr_dist
    
    #Add to closest distance to list
    distance_station.append(closest_dist)

[-37.9087147, 144.7480729]

In [6]:
#Calculate distance to CBD

import openrouteservice
import pandas as pd
import numpy as np

#Get API key from local device so client can't steal my data
with open('../keys/ors_api_key.txt') as f:
    API_KEY = f.read()
    print(API_KEY)

client = openrouteservice.Client(key=API_KEY) 

CBD_COORDINATES = [-37.8136, 144.9631]
distance_cbd = []

#Find distance to CBD for each property
for index, row in property_df.iterrows():

    coords = ((property_df["coordinates1"][index][1], property_df["coordinates1"][index][0]),(CBD_COORDINATES[1], CBD_COORDINATES[0]))
    routes = client.directions(coords, radiuses=-1)
    curr_dist = routes["routes"][0]["segments"][0]["distance"]

    #Add distance to list
    distance_cbd.append(curr_dist)

5b3ce3597851110001cf6248f3d5d4cc6e1040bdbdb3f91f8485a47d
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye
ye


KeyboardInterrupt: 

In [17]:
## Path name for train station distance dataset
#'../data/curated/bed1.txt'
#'../data/curated/bed2.txt'
#'../data/curated/bed3.txt'
#'../data/curated/bed4.txt'
#'../data/curated/bed5.txt'
#'../data/curated/featured.txt'
#'../data/curated/newest.txt'
#'../data/curated/highest_price.txt'
#'../data/curated/lowest_price.txt'

## Path name for CBD distance dataset
#'../data/curated/bed1_distance_cbd.txt'
#'../data/curated/bed2_distance_cbd.txt'
#'../data/curated/bed3_distance_cbd.txt'
#'../data/curated/bed4_distance_cbd.txt'
#'../data/curated/bed5_distance_cbd.txt'
#'../data/curated/featured_distance_cbd.txt' 
#'../data/curated/newest_distance_cbd.txt' 
#'../data/curated/highest_distance_cbd.txt'
#'../data/curated/lowest_distance_cbd.txt'

#Save distance as txt file
with open("../data/curated/bed1.txt", "w") as output:
    output.write(str(distance_cbd))
